In [1]:
import torch
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image

In [2]:
import torchvision.transforms as T

T_norm = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])

T_tensor = T.Compose([
    T.ToTensor()
])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def getPredict(a):
    pred =a['out'].data.cpu().numpy()
    return np.argmax(pred, axis=1)[0]

def get_segment_labels(image, model, device, transform):
    # transform the image to tensor and load into computation device
    image = transform(image).to(device)
    image = image.unsqueeze(0) # add a batch dimension
    outputs = model(image)
    return outputs

def getSP(mask):
    return np.sum(mask==1)/(mask.shape[0]*mask.shape[1])

def getOSR(mask):
    return np.sum(mask==2)/(mask.shape[0]*mask.shape[1])

In [3]:
num_class = 3
confusion_matrix = np.zeros((num_class,)*2)

def Pixel_Accuracy(confusion_matrix):
    Acc = np.diag(confusion_matrix).sum() / confusion_matrix.sum()
    return Acc

def Pixel_Accuracy_Class(confusion_matrix):
    Acc = np.diag(confusion_matrix) / confusion_matrix.sum(axis=1)
    #Acc = np.nanmean(Acc)
    return Acc

def Mean_Intersection_over_Union(confusion_matrix):
    MIoU = np.diag(confusion_matrix) / (
                np.sum(confusion_matrix, axis=1) + np.sum(confusion_matrix, axis=0) -
                np.diag(confusion_matrix))
    MIoU = np.nanmean(MIoU)
    return MIoU

def Frequency_Weighted_Intersection_over_Union(confusion_matrix):
    freq = np.sum(confusion_matrix, axis=1) / np.sum(confusion_matrix)
    iu = np.diag(confusion_matrix) / (
                np.sum(confusion_matrix, axis=1) + np.sum(confusion_matrix, axis=0) -
                np.diag(confusion_matrix))

    FWIoU = (freq[freq > 0] * iu[freq > 0]).sum()
    return FWIoU

def _generate_matrix(num_class, gt_image, pre_image):
    mask = (gt_image >= 0) & (gt_image < num_class)
    label = num_class * gt_image[mask].astype('int') + pre_image[mask]
    count = np.bincount(label, minlength=num_class**2)
    confusion_matrix = count.reshape(num_class, num_class)
    return confusion_matrix

In [4]:
def add_batch(confusion_matrix, num_class, gt_image, pre_image):
    #assert gt_image.shape == pre_image.shape
    cm = confusion_matrix + _generate_matrix(num_class, gt_image, pre_image)
    return cm

def reset(num_class):
    return np.zeros((num_class,) * 2)

In [5]:
dict_Md = {'180920':500,
'180926':586,
'181009':747,
'181015':828,
'181022':903,
}

dict_Pr = {'180919':579,
'180926':696,
'181010':895,
'181016':989,
'181116':1295,
'190913':324,
'190927':565,
'191007':706,
'191029':981,
'20200916':570,
'20200917':570,
'20200924':725,
'20200930':785,
'20201008':873,
'20201014':932
}

In [98]:
# Load the trained model 
model = torch.load(r'C:\Users\A60026184\Desktop\ModelB\Results\weights_cascBaug5_v0.pt')
model.eval()

print('DONE Loading models...')

DONE Loading models...


In [95]:
img_files = glob(r'C:\Users\A60026184\Desktop\ModelB\Test\Rapeseed\ImgMasked\*.*')
msk_path = r'C:\Users\A60026184\Desktop\ModelB\Test\Rapeseed\Join'
pathout = r'H:\Aurélie\ModelEval\ModelB\Raw\B1'

In [ ]:
cm = reset(3)

for f in img_files:
    filename = f.split('\\')[-1] #[:8] #Model B full
    #if filename[0]=='0':
    #    filename = filename[1:]
    msk = glob(msk_path+'\\'+filename)[0]
    
    # read the image
    image = Image.open(f)
    # Read  a sample image and mask from the data-set
    mask = np.array(Image.open(msk))
    
    ### ----------- MODEL B ------------ ###
    mask[mask < 127] = 0
    mask[mask > 127] = 1
    mask[mask[:, :, 1] > 0] = 2
    mask = mask[:,:,0]
    
    result = get_segment_labels(image, model, device, T_norm)
    pred = getPredict(result)
    
    cm += _generate_matrix(3, mask, pred)

Acc_a = Pixel_Accuracy(cm)
cAcc_a = Pixel_Accuracy_Class(cm)
mIoU = Mean_Intersection_over_Union(cm)
fwIoU = Frequency_Weighted_Intersection_over_Union(cm)

print(' Pixel Accuracy is', round(Acc_a*100,2), '%\n',
      'Class Accuracy \n', 
      'Background ', round(cAcc_a[0]*100,2),
      '%\n Vegetation', round(cAcc_a[1]*100,2),
      ### ----------- MODEL B ------------ ###
      '%\n Oilseed rape', round(cAcc_a[2]*100,2),
      '%\n mean IoU is', round(mIoU*100,2),
      '%\n frequency weighted IoU is ',round(fwIoU*100,2),'%')
    

In [99]:
SP_gt = []
SP_pred = []
OSR_gt = []
OSR_pred = []
dat, loc, adv, mod, rep = [], [], [], [], []

for f in img_files:
    
    filename = f.split('\\')[-1]
    #if filename[0]=='0':
    #    filename=filename[1:]
    msk = glob(msk_path+'\\'+filename)[0]
    
    filename = filename.split('.')[0]
    
    dat.append(filename.split('_')[1])
    loc.append(filename.split('_')[2])
    adv.append(filename.split('_')[3])
    mod.append(filename.split('_')[4])
    rep.append(filename.split('_')[5])
    
    # read the image
    image = Image.open(f)
    # Read  a sample image and mask from the data-set
    mask = np.array(Image.open(msk))
    mask[mask < 127] = 0
    mask[mask >= 127] = 1
    mask[mask[:, :, 1] > 0] = 2
    mask = mask[:,:,0]
    
    result = get_segment_labels(image, model, device, T_norm)
    pred = getPredict(result)  
    
    SP_gt.append(getSP(mask))
    SP_pred.append(getSP(pred))
    
    OSR_gt.append(getOSR(mask))
    OSR_pred.append(getOSR(pred))

In [92]:
filename

'9_20200916_Ch_A_M02_4'

In [100]:
import pandas as pd
df_SPgt = pd.DataFrame(SP_gt,columns = ['Gt SP'])
df_OSRgt = pd.DataFrame(OSR_gt,columns = ['Gt OSR'])

SP_err = np.abs(np.array(SP_gt)-np.array(SP_pred))
OSR_err = np.abs(np.array(OSR_gt)-np.array(OSR_pred))
df_SP = pd.DataFrame(SP_err, columns=['SP_err'])
df_OSR = pd.DataFrame(OSR_err, columns=['OSR_err'])

DJ = np.zeros(np.shape(SP_err))
df_DJ = pd.DataFrame(DJ, columns=['DJ'])


df_dat = pd.DataFrame(dat, columns=['Date'])
df_loc = pd.DataFrame(loc, columns=['Lieu'])
df_adv = pd.DataFrame(adv, columns=['Type'])
df_mod = pd.DataFrame(mod, columns=['Moda'])
df_rep = pd.DataFrame(rep, columns=['Rep'])

df = pd.concat([df_dat, df_loc, df_adv, df_mod, df_rep,
                df_SPgt, df_SP, df_OSRgt, df_OSR, df_DJ], axis=1)
df = df.groupby(['Date', 'Lieu', 'Type', 'Moda', 'Rep']).mean().reset_index()

df['DJ'][df['Lieu']=='Md'] = df['Date'][df['Lieu']=='Md'].map(dict_Md)
df['DJ'][df['Lieu']!='Md'] = df['Date'][df['Lieu']!='Md'].map(dict_Pr)

df.to_csv(pathout + '\\rapeseed_Bv0.csv')

<ipython-input-100-5ccab35563f3>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DJ'][df['Lieu']=='Md'] = df['Date'][df['Lieu']=='Md'].map(dict_Md)
<ipython-input-100-5ccab35563f3>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DJ'][df['Lieu']!='Md'] = df['Date'][df['Lieu']!='Md'].map(dict_Pr)
